In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()  
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()  
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) - 0)**2).cuda() 
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 1.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Neumann.npy", error_save)
np.save("DRM_relative_error_4D_Neumann.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(22.3845, device='cuda:0')
current relative error is:  tensor(1.8770, device='cuda:0')
current epoch is:  50
current loss is:  tensor(-5.4286, device='cuda:0')
current relative error is:  tensor(0.8782, device='cuda:0')
current epoch is:  100
current loss is:  tensor(-11.1363, device='cuda:0')
current relative error is:  tensor(0.5329, device='cuda:0')
current epoch is:  150
current loss is:  tensor(-12.6912, device='cuda:0')
current relative error is:  tensor(0.4268, device='cuda:0')
current epoch is:  200
current loss is:  tensor(-13.7510, device='cuda:0')
current relative error is:  tensor(0.3795, device='cuda:0')
current epoch is:  250
current loss is:  tensor(-15.0891, device='cuda:0')
current relative error is:  tensor(0.3354, device='cuda:0')
current epoch is:  300
current loss is:  tensor(-14.0208, device='cuda:0')
current relative error is:  tensor(0.3108, device='cuda:0')
current epoch is:  350
current loss is:  tensor(-14.1290, de

current epoch is:  3050
current loss is:  tensor(-20.6239, device='cuda:0')
current relative error is:  tensor(0.0399, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(-19.0000, device='cuda:0')
current relative error is:  tensor(0.0270, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-17.6613, device='cuda:0')
current relative error is:  tensor(0.0251, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-19.5618, device='cuda:0')
current relative error is:  tensor(0.0307, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-20.7229, device='cuda:0')
current relative error is:  tensor(0.0360, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-17.5716, device='cuda:0')
current relative error is:  tensor(0.0570, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-20.0264, device='cuda:0')
current relative error is:  tensor(0.0561, device='cuda:0')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-21.9949, device='cuda:0')
current relative error is:  tensor(0.0435, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(-21.2135, device='cuda:0')
current relative error is:  tensor(0.0392, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-19.9425, device='cuda:0')
current relative error is:  tensor(0.0339, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-19.5293, device='cuda:0')
current relative error is:  tensor(0.0823, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-19.7003, device='cuda:0')
current relative error is:  tensor(0.0265, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-20.5551, device='cuda:0')
current relative error is:  tensor(0.0163, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-20.2432, device='cuda:0')
current relative error is:  tensor(0.0280, device='cuda:0')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-19.4856, device='cuda:0')
current relative error is:  tensor(0.0454, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(-19.1773, device='cuda:0')
current relative error is:  tensor(0.0216, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-20.3650, device='cuda:0')
current relative error is:  tensor(0.0148, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-20.7775, device='cuda:0')
current relative error is:  tensor(0.0259, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-20.8554, device='cuda:0')
current relative error is:  tensor(0.0588, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-17.4755, device='cuda:0')
current relative error is:  tensor(0.0111, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-20.2246, device='cuda:0')
current relative error is:  tensor(0.0235, device='cuda:0')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-19.7782, device='cuda:0')
current relative error is:  tensor(0.0274, device='cuda:0')
current epoch is:  12200
current loss is:  tensor(-19.4864, device='cuda:0')
current relative error is:  tensor(0.0344, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(-19.8815, device='cuda:0')
current relative error is:  tensor(0.0439, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-20.1307, device='cuda:0')
current relative error is:  tensor(0.0141, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-19.9971, device='cuda:0')
current relative error is:  tensor(0.0582, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-20.3733, device='cuda:0')
current relative error is:  tensor(0.0298, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-19.8273, device='cuda:0')
current relative error is:  tensor(0.0172, device='cuda:0')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-19.7887, device='cuda:0')
current relative error is:  tensor(0.0190, device='cuda:0')
current epoch is:  15200
current loss is:  tensor(-20.0158, device='cuda:0')
current relative error is:  tensor(0.0193, device='cuda:0')
current epoch is:  15250
current loss is:  tensor(-20.6487, device='cuda:0')
current relative error is:  tensor(0.0267, device='cuda:0')
current epoch is:  15300
current loss is:  tensor(-17.7813, device='cuda:0')
current relative error is:  tensor(0.0547, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(-17.7574, device='cuda:0')
current relative error is:  tensor(0.0344, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(-21.5583, device='cuda:0')
current relative error is:  tensor(0.0220, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(-20.0886, device='cuda:0')
current relative error is:  tensor(0.0244, device='cuda:0')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-20.2384, device='cuda:0')
current relative error is:  tensor(0.0232, device='cuda:0')
current epoch is:  18200
current loss is:  tensor(-20.4777, device='cuda:0')
current relative error is:  tensor(0.0487, device='cuda:0')
current epoch is:  18250
current loss is:  tensor(-20.8593, device='cuda:0')
current relative error is:  tensor(0.0419, device='cuda:0')
current epoch is:  18300
current loss is:  tensor(-18.6588, device='cuda:0')
current relative error is:  tensor(0.0252, device='cuda:0')
current epoch is:  18350
current loss is:  tensor(-21.0422, device='cuda:0')
current relative error is:  tensor(0.0219, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(-19.3195, device='cuda:0')
current relative error is:  tensor(0.0291, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(-20.8827, device='cuda:0')
current relative error is:  tensor(0.0459, device='cuda:0')
current epoch is:  18500
current loss is:

In [18]:
x_test = torch.rand(10, 4)
torch.abs(model(x) - u_ex(x))

tensor([[0.0142],
        [0.0331],
        [0.0016],
        ...,
        [0.0192],
        [0.0210],
        [0.0002]], device='cuda:0', grad_fn=<AbsBackward>)